In [1]:
import ee
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [2]:
# If it's for the first time, authentication is required before running this code
# https://developers.google.com/earth-engine/guides/python_install-conda#windows_5
ee.Initialize()

In [3]:
# earth engine location of the subsets of the 10km grid created separately in the script "00_create_10_by_10_grid.R"
# 10km grid shapefile needs to be uploaded to assets in your GEE before running this line
grid = ee.FeatureCollection("projects/ee-akawano/assets/grid_10km")

# google drive associated with your GEE account
output_drive = "elevation_10km"
timezone = "utc"

In [4]:
# NASA SRTM Digital Elevation 30m, 2000
# https://developers.google.com/earth-engine/datasets/catalog/USGS_SRTMGL1_003#bands

maiac = ee.Image("USGS/SRTMGL1_003").select("elevation")
maiac_proj = maiac.projection()
pixel_res = maiac_proj.nominalScale().getInfo()

In [8]:
maiac_images = ee.ImageCollection(maiac)
maiac_images_reduced = maiac_images.map(lambda daily_im: (
        daily_im.reduceRegions(
            collection = grid,
            reducer = ee.Reducer.mean(),
            crs = maiac_proj,
            scale = pixel_res
        )))
grid_maiac = ee.FeatureCollection(maiac_images_reduced).flatten()

In [9]:
# make a list of properties to export 
export_properties = ["grid_id", "mean"]
export_properties

['grid_id', 'mean']

In [10]:
output_name = "elevation_10km_2000"
elevation_task = ee.batch.Export.table.toDrive(
        collection = grid_maiac,
        folder = output_drive,
        description = output_name,
        fileFormat = "CSV",
        selectors = export_properties)
print(f"saving file as", {output_name})
elevation_task.start()

saving file as {'elevation_10km_2000'}
